In [5]:
import dowhy
import networkx as nx
import pandas as pd
import numpy as np

In [125]:
n_samples = 1000
random_sample = lambda: np.random.normal(loc = 10, size=n_samples) 
random_binary_sample = lambda p=0.5: np.random.choice([0, 1], size=n_samples, p=(1-p, p)).astype(bool)

biased_random_binary_sample = lambda cond, p1, p2: np.where(cond, random_binary_sample(p1), random_binary_sample(p2))

_ = random_binary_sample()
_2 = biased_random_binary_sample(_, 0.3, 0.8)

df = pd.DataFrame({
    "x_age": random_sample(),
    "x_sex": _,
    "d_d0": random_binary_sample(),
    "d_d1": _2,
    "y_K": random_sample(),
    "y_A": random_sample() + 2 * _2 - 1.67 * _
})

In [195]:
def get_diagnoses(df):
    return [d for d in df.columns if d.startswith("d_")]

def get_other_covariates(df):
    return [x for x in df.columns if x.startswith("x_")]

def get_outcome(df):
    return [y for y in df.columns if y.startswith("y_")]


def construct_network_indep_diagnosis(df):
    x_to_y = [(x, y) for x in get_other_covariates(df) for y in get_outcome(df)]
    x_to_d = [(x, d) for x in get_other_covariates(df) for d in get_diagnoses(df)]
    d_to_y = [(d, y) for d in get_diagnoses(df) for y in get_outcome(df)]
    
    return str(nx.nx_pydot.to_pydot(nx.DiGraph(x_to_y + x_to_d + d_to_y)))


def construct_network_full(df):
    all_covariates = get_other_covariates(df) + get_diagnoses(df)
    return str(nx.nx_pydot.to_pydot(nx.DiGraph(
        [(x, y) for x in all_covariates for y in all_covariates] + [(x, y) for x in all_covariates for y in get_outcome(df)]
    )))



def construct_network_indep_diagnosis_to_risk(df):
    x_to_y = [(x, y) for x in get_other_covariates(df) for y in get_outcome(df)]
    x_to_d = [(x, d) for x in get_other_covariates(df) for d in get_diagnoses(df)]
    d_to_y = [(d, y) for d in get_diagnoses(df) for y in get_outcome(df)]
    y_to_risk = [(y, "Risk") for y in get_outcome(df)]
    
    return str(nx.nx_pydot.to_pydot(nx.DiGraph(x_to_y + x_to_d + d_to_y + y_to_risk)))


# Playground

In [277]:
# m = dowhy.CausalModel(df, treatment=get_diagnoses(df), outcome=get_outcome(df), graph=construct_network_indep_diagnosis(df))
# m = dowhy.CausalModel(df, treatment=["d_d0", "d_d1"], outcome="y_A", graph=construct_network_indep_diagnosis(df))
# m = dowhy.CausalModel(df, treatment=["d_d0", "d_d1"], outcome="y_A", graph=construct_network_indep_diagnosis(df))
m = dowhy.CausalModel(df, treatment="d_d1", outcome="y_A", graph=construct_network_indep_diagnosis(df))
# m = dowhy.CausalModel(df, treatment=["d_d0", "d_d1"], outcome="y_A", graph=construct_network_full(df))

In [278]:
identified = m.identify_effect(proceed_when_unidentifiable=True)
# print(identified)

estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True, confidence_intervals=True)
# estimate = m.estimate_effect(identified, method_name="backdoor.distance_matching") #, test_significance=True)

print("\n\n===")
print(estimate)

print("\n\n===")
# ref = m.refute_estimate(identified, estimate, method_name="random_common_cause")
# ref = m.refute_estimate(identified, estimate, method_name="add_unobserved_common_cause")
# print(ref.refutation_result)

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_va

In [188]:
# estimate.get_confidence_intervals()
# estimate.test_stat_significance()
# estimate.params
# estimate.estimate_conditional_effects("d_d1")
estimate.value

# Data Analysis

In [238]:
def causal_estimate_independent_diagnoses(df, graph, outcome):
    ds = get_diagnoses(df)[:5]
    
    effects = {}
    
    for diagnosis in ds:
        m = dowhy.CausalModel(df, treatment=diagnosis, outcome=outcome, graph=graph)
        
        identified = m.identify_effect(proceed_when_unidentifiable=True)
        # print(identified)

        estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True, confidence_intervals=True)
        # estimate = m.estimate_effect(identified, method_name="backdoor.distance_matching") #, test_significance=True)

        # print("===")
        # print(estimate)
        
        effects[diagnosis] = (estimate, estimate.value)
    
    return effects

In [ ]:
def causal_estimate_independent_diagnoses(df, graph, outcome):
    ds = get_diagnoses(df)
    
    effects = {}
    
    for diagnosis in ds:
        m = dowhy.CausalModel(df, treatment=diagnosis, outcome=outcome, graph=graph)
        
        identified = m.identify_effect(proceed_when_unidentifiable=True)
        print(identified)

        estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True)
        # estimate = m.estimate_effect(identified, method_name="backdoor.distance_matching") #, test_significance=True)

        # print("===")
        # print(estimate)
        
        effects[diagnosis].append((estimate, estimate.value))
    
    return effects

In [275]:
real_df


ckd = "N18.3"
hyper = "I10"

In [354]:
# real_df = pd.read_csv("../data/honza_data.csv")
real_df = pd.read_csv("../data/honza_jirka_data.csv")
# print(get_diagnoses(real_df)[:10] + get_outcome(real_df) + get_other_covariates(real_df))
# i = 
# real_df = real_df[get_diagnoses(real_df)[i:i+1] + get_outcome(real_df) + get_other_covariates(real_df)]
# real_df = real_df[list(filter(lambda x: ckd in x, get_diagnoses(real_df))) + get_outcome(real_df) + get_other_covariates(real_df)]
real_df = real_df[get_diagnoses(real_df) + get_outcome(real_df) + get_other_covariates(real_df)]
real_df = real_df.loc[~real_df.y_ACR.isna()]
# real_df.head()

In [305]:
list(filter(lambda x: hyper in x, get_diagnoses(real_df)))
real_df.loc[real_df["d_N18.3"]]
real_df[get_diagnoses(real_df)].sum().sum()

0

In [310]:
real_df.x_sex.iloc[100:]

448      F
450      F
454      F
459      F
460      F
        ..
61357    M
61363    M
61367    M
61405    M
61411    M
Name: x_sex, Length: 10282, dtype: object

In [328]:
def mmoll_to_mgL(x):
    return x * 0.0113096584483149

def populate_eGFR(df):
    """
    https://www.tuftsmedicalcenter.org/-/media/Brochures/TuftsMC/Research-Clinical-Trials/Implementation-of-2021-CKD-EPI-Equations-15-Oct-2021-b.ashx?la=en&hash=747F5F741555F2A8FC04720D32AC97C1B55CF240
    """
    creatinin_mgL = mmoll_to_mgL(df.y_KREA)
    female = df.x_sex == "F"
    
    common = 142 * .9938 ** df.x_age * np.where(female.values, 1.012, 1.0)
    kappa = np.where(female, 0.7, 0.9)
    exponent = - np.where((creatinin_mgL <= kappa).values, np.where(female.values, .241, .302), 1.2)
    
    eGFR = common * (creatinin_mgL / kappa) ** exponent
    
    df = df.copy()
    df["y_eGFR"] = eGFR
    return df

In [351]:
real_df = populate_eGFR(real_df)
(d.y_eGFR < 15).mean()

In [353]:
# graph = construct_network_indep_diagnosis(real_df)
# real_df.to_csv("../data/honza_jirka_martin_data.csv", index=False)

In [347]:
# m = dowhy.CausalModel(real_df, treatment=get_diagnoses(real_df), outcome="y_ACR", graph=construct_network_indep_diagnosis(real_df))

treatments = ["d_I10"]
w_df = real_df[treatments + get_other_covariates(real_df) + get_outcome(real_df)]
m = dowhy.CausalModel(w_df, treatment=treatments, outcome="y_ACR", graph=construct_network_indep_diagnosis(w_df))
        
# identified = m.identify_effect(proceed_when_unidentifiable=True)
identified = m.identify_effect(proceed_when_unidentifiable=False)
print(identified)

estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True, confidence_intervals=True)
print("\n\n\n===========")
print(estimate)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                
────────(E[y_ACR|x_BMI,x_sex,x_age])
d[d_I10]                            
Estimand assumption 1, Unconfoundedness: If U→{d_I10} and U→y_ACR then P(y_ACR|d_I10,x_BMI,x_sex,x_age,U) = P(y_ACR|d_I10,x_BMI,x_sex,x_age)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': []}



*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                
────────(E[y_ACR|x_BMI,x_sex,x_age])
d[d_I10]                            
Estimand assumption 1, Unconfoundedness: If U→{d_I10} and U→

In [ ]:
a = estimate.get_confidence_intervals()
a[(a > 50).all(axis=1)]
# (a > 0.2).sum(axis=1)

In [337]:
real_df.y_ACR.describe()

count    10382.000000
mean        12.525905
std         41.425530
min          0.000000
25%          0.700000
50%          1.600000
75%          5.900000
max        865.800000
Name: y_ACR, dtype: float64

causal_estimate_independent_diagnoses(real_df, graph, "y_ACR")

In [286]:
real_df

,d_J30.2,d_J30.3,d_J45.8,d_I10,d_I25.9,d_Z95.1,d_U58.9,d_E78.2,d_E11.9,d_H40.9,y_KREA,y_ACR,x_age,x_BMI,x_sex
8,False,False,False,False,False,False,False,False,False,False,76.8,0.5,28.1,17.6,NaN
15,False,False,False,False,False,False,False,False,False,False,87.0,0.5,52.0,38.4,NaN
43,False,False,False,False,False,False,False,False,False,False,67.1,0.6,32.2,22.9,NaN
47,False,False,False,False,False,False,False,False,False,False,61.8,0.7,23.6,19.8,NaN
53,False,False,False,False,False,False,False,False,False,False,81.4,0.4,49.5,24.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61357,False,False,False,False,False,False,False,False,False,False,89.5,1.6,58.4,38.2,M
61363,False,False,False,False,False,False,False,False,False,False,120.2,1.0,59.3,32.1,M
61367,False,False,False,False,False,False,False,False,False,False,73.3,0.8,65.5,35.9,M
61405,False,False,False,False,False,False,False,False,False,False,70.5,0.8,23.4,27.3,M


In [288]:
#real_df.head()
real_df.describe(include="all")

,d_J30.2,d_J30.3,d_J45.8,d_I10,d_I25.9,d_Z95.1,d_U58.9,d_E78.2,d_E11.9,d_H40.9,y_KREA,y_ACR,x_age,x_BMI,x_sex
count,10382,10382,10382,10382,10382,10382,10382,10382,10382,10382,10377.000000,10382.000000,10382.000000,10382.000000,10212
unique,1,1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,2
top,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,M
freq,10382,10382,10382,10382,10382,10382,10382,10382,10382,10382,NaN,NaN,NaN,NaN,5440
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.624313,12.525905,59.618821,28.406136,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.253719,41.425530,16.240780,5.974171,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.100000,0.000000,16.300000,12.600000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.000000,0.700000,48.100000,24.500000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.100000,1.600000,61.500000,27.800000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.500000,5.900000,72.300000,31.600000,NaN


In [243]:
real_df.describe()

,y_KREA,y_ACR,x_age,x_BMI
count,10377.000000,10382.000000,10382.000000,10382.000000
mean,119.624313,12.525905,59.618821,28.406136
std,96.253719,41.425530,16.240780,5.974171
min,28.100000,0.000000,16.300000,12.600000
25%,75.000000,0.700000,48.100000,24.500000
50%,93.100000,1.600000,61.500000,27.800000
75%,127.500000,5.900000,72.300000,31.600000
max,1363.400000,865.800000,100.200000,227.200000
